In [1]:
import pandas as pd
import logging
import numpy as np
from scipy.stats import norm
import utils
import importlib

importlib.reload(utils)

<module 'utils' from '/Users/mattgc/code/match-importance/utils.py'>

In [2]:
inp = input("Level of logging (DEBUG, INFO, WARNING, ERROR, CRITICAL): ")
logging.basicConfig(level=getattr(logging, inp), force=True)

# Import Data

In [3]:
data = pd.read_csv('data/all_years/24_25.csv')
data["Date"] = pd.to_datetime(data["Date"], format="%d/%m/%Y")
data = data.sort_values(by=["Date", "Time"]).reset_index(drop=True)
t = len(data)
data

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA
0,E0,2024-08-16,20:00,Man United,Fulham,1,0,H,0,0,...,1.86,2.07,1.83,2.11,1.88,2.11,1.82,2.05,1.90,2.08
1,E0,2024-08-17,12:30,Ipswich,Liverpool,0,2,A,0,0,...,2.05,1.88,2.04,1.90,2.20,2.00,1.99,1.88,2.04,1.93
2,E0,2024-08-17,15:00,Arsenal,Wolves,2,0,H,1,0,...,2.02,1.91,2.00,1.90,2.05,1.93,1.99,1.87,2.02,1.96
3,E0,2024-08-17,15:00,Everton,Brighton,0,3,A,0,1,...,1.87,2.06,1.86,2.07,1.92,2.10,1.83,2.04,1.88,2.11
4,E0,2024-08-17,15:00,Newcastle,Southampton,1,0,H,1,0,...,1.87,2.06,1.88,2.06,1.89,2.10,1.82,2.05,1.89,2.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,E0,2024-12-04,19:30,Southampton,Chelsea,1,5,A,1,3,...,2.07,1.86,2.07,1.87,2.07,1.90,2.01,1.86,2.07,1.92
136,E0,2024-12-04,20:15,Arsenal,Man United,2,0,H,0,0,...,2.07,1.86,2.07,1.86,2.07,1.93,1.99,1.87,2.06,1.93
137,E0,2024-12-04,20:15,Aston Villa,Brentford,3,1,H,3,0,...,1.85,2.08,1.85,2.08,1.88,2.08,1.84,2.02,1.92,2.07
138,E0,2024-12-05,19:30,Fulham,Brighton,3,1,H,1,0,...,1.94,1.99,1.94,1.98,1.96,2.00,1.92,1.96,1.97,2.01


In [4]:
remaining = pd.read_csv('data/all_years/24_25_remaining.csv')
remaining

,HomeTeam,AwayTeam
0,Everton,Liverpool
1,Aston Villa,Southampton
2,Brentford,Newcastle
3,Crystal Palace,Man City
4,Man Utd,Nott'm Forest
...,...,...
235,Newcastle United,Everton
236,Nottingham Forest,Chelsea
237,Southampton,Arsenal
238,Spurs,Brighton


In [5]:
# Concatenate remaining data to the original data
data = pd.concat([data, remaining], ignore_index=True)
data["TeamEloWinProb"] = 0
data

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA,TeamEloWinProb
0,E0,2024-08-16,20:00,Man United,Fulham,1.0,0.0,H,0.0,0.0,...,2.07,1.83,2.11,1.88,2.11,1.82,2.05,1.90,2.08,0
1,E0,2024-08-17,12:30,Ipswich,Liverpool,0.0,2.0,A,0.0,0.0,...,1.88,2.04,1.90,2.20,2.00,1.99,1.88,2.04,1.93,0
2,E0,2024-08-17,15:00,Arsenal,Wolves,2.0,0.0,H,1.0,0.0,...,1.91,2.00,1.90,2.05,1.93,1.99,1.87,2.02,1.96,0
3,E0,2024-08-17,15:00,Everton,Brighton,0.0,3.0,A,0.0,1.0,...,2.06,1.86,2.07,1.92,2.10,1.83,2.04,1.88,2.11,0
4,E0,2024-08-17,15:00,Newcastle,Southampton,1.0,0.0,H,1.0,0.0,...,2.06,1.88,2.06,1.89,2.10,1.82,2.05,1.89,2.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,NaN,NaT,NaN,Newcastle United,Everton,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
376,NaN,NaT,NaN,Nottingham Forest,Chelsea,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
377,NaN,NaT,NaN,Southampton,Arsenal,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
378,NaN,NaT,NaN,Spurs,Brighton,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [6]:
to_predict = pd.read_csv('data/epl-test.csv')
to_predict

,Date,HomeTeam,AwayTeam
0,01-Feb-25,AFC Bournemouth,Liverpool
1,01-Feb-25,Arsenal,Man City
2,01-Feb-25,Brentford,Spurs
3,01-Feb-25,Chelsea,West Ham
4,01-Feb-25,Everton,Leicester City
5,01-Feb-25,Ipswich Town,Southampton
6,01-Feb-25,Man Utd,Crystal Palace
7,01-Feb-25,Newcastle,Fulham
8,01-Feb-25,Nottingham Forest,Brighton
9,01-Feb-25,Wolves,Aston Villa


In [7]:
t_k = data[(data["HomeTeam"] == "AFC Bournemouth") & (data["AwayTeam"] == "Liverpool")].index[0]
print(t_k)
data.iloc[t_k]

230


Div                           NaN
Date                          NaT
Time                          NaN
HomeTeam          AFC Bournemouth
AwayTeam                Liverpool
                       ...       
AvgCAHH                       NaN
AvgCAHA                       NaN
BFECAHH                       NaN
BFECAHA                       NaN
TeamEloWinProb                  0
Name: 230, Length: 121, dtype: object

# Run a single simulation

In [8]:
standings = utils.calculate_standings(data, 0, t)
standings

,Team,Points
0,Liverpool,35
1,Arsenal,28
2,Chelsea,28
3,Man City,26
4,Brighton,23
5,Fulham,22
6,Nott'm Forest,22
7,Aston Villa,22
8,AFC Bournemouth,21
9,Newcastle,20


In [9]:
simulated_standings = utils.run_simulation(data, standings, t, t_k).sort_values(by="Points", ascending=False).reset_index(drop=True)
simulated_standings

,Team,Points
0,Man City,67
1,Chelsea,66
2,Liverpool,64
3,AFC Bournemouth,60
4,Arsenal,60
5,Fulham,53
6,Crystal Palace,52
7,Aston Villa,51
8,Brighton,50
9,Brentford,48


In [10]:
utils.calculate_outcomes(simulated_standings, "AFC Bournemouth", "Liverpool")

(0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1)

# Predict match importance for match $t+k$ with repeated simulations

In [11]:
data.loc[t_k, ["HI", "AI"]] = utils.calculate_match_importance( # type: ignore
    data,
    standings,
    t_k,
    t,
    nruns=100,
)

In [12]:
data.iloc[t_k]

Div                           NaN
Date                          NaT
Time                          NaN
HomeTeam          AFC Bournemouth
AwayTeam                Liverpool
                       ...       
BFECAHH                       NaN
BFECAHA                       NaN
TeamEloWinProb                  0
HI                            0.0
AI                           0.03
Name: 230, Length: 123, dtype: object

# Backfill match importance in master file

In [13]:
master = pd.read_csv('data/michal-master.csv').drop_duplicates()
master["Date"] = pd.to_datetime(master["Date"], format="%Y-%m-%d").dt.strftime("%d-%b-%y")
master

,Date,HomeTeam,AwayTeam,HomeElo,AwayElo,HomeRank,AwayRank,HomePlayerRatingCount,HomePlayerRatingMean,HomePlayerRatingStd,AwayPlayerRatingCount,AwayPlayerRatingMean,AwayPlayerRatingStd,TeamEloWinProb,PlayerEloDrawProb,PlayerEloWinProb,HomeEloMomentum,AwayEloMomentum,HomeRankMomentum,AwayRankMomentum
0,05-Jan-10,Stoke,Fulham,1621.513306,1739.930908,15.0,8.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.335889,0.330000,0.330000,0.000000,0.000000,0.0,0.0
1,09-Jan-10,Arsenal,Everton,1877.290039,1703.894897,3.0,9.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.730693,0.330000,0.330000,0.000000,0.000000,0.0,0.0
2,09-Jan-10,Birmingham,Man United,1653.174805,1927.595947,11.0,2.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.170839,0.330000,0.330000,0.000000,0.000000,0.0,0.0
3,11-Jan-10,Man City,Blackburn,1745.521851,1642.066650,7.0,13.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.644634,0.330000,0.330000,0.000000,0.000000,0.0,0.0
4,16-Jan-10,Chelsea,Sunderland,1945.430176,1606.012695,1.0,18.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.875865,0.330000,0.330000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,19-May-24,Crystal Palace,Aston Villa,1742.572754,1787.570190,9.0,7.0,14.0,6.966429,0.327689,14.0,6.925714,0.291908,0.435603,0.269324,0.475556,-48.135376,41.060791,6.0,-2.0
5800,19-May-24,Liverpool,Wolves,1898.975098,1681.206665,3.0,16.0,17.0,7.169412,0.293948,15.0,6.921333,0.225325,0.777918,0.209490,0.643965,19.503418,38.082886,0.0,-3.0
5801,19-May-24,Luton,Fulham,1577.058716,1707.895386,22.0,13.0,16.0,6.679375,0.278242,15.0,6.836000,0.187111,0.320134,0.279518,0.298134,39.452759,11.820312,0.0,0.0
5802,19-May-24,Man City,West Ham,2044.003174,1728.072998,1.0,11.0,13.0,7.429231,0.306104,12.0,6.951667,0.301160,0.860403,0.117857,0.817173,-3.334229,51.946533,0.0,-3.0


In [14]:
all_years = pd.concat(
    [
        pd.read_csv(f"data/all_years/{season}.csv")
        for season in [
            f"{i}_{i+1}"
            for i in range(17, 24)
        ]
    ]
)
all_years["Date"] = pd.to_datetime(all_years["Date"], format="%d/%m/%Y").dt.strftime("%d-%b-%y")
all_years = all_years.sort_values(by=["Date"]).reset_index(drop=True)
all_years

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,01-Apr-18,Chelsea,Tottenham,1,3,A,1,1,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,01-Apr-18,Arsenal,Stoke,3,0,H,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,01-Apr-19,Arsenal,Newcastle,2,0,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,01-Apr-23,Chelsea,Aston Villa,0,2,A,0,1,A,...,1.89,-1.00,2.04,1.86,2.07,1.86,2.11,1.88,2.05,1.82
4,E0,01-Apr-23,Brighton,Brentford,3,3,D,2,2,D,...,2.03,-1.00,2.01,1.89,2.10,1.84,2.11,1.90,2.05,1.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,E0,31-Oct-20,Sheffield United,Man City,0,1,A,0,1,A,...,2.47,1.50,2.05,1.85,2.06,1.88,2.09,1.91,2.04,1.85
2656,E0,31-Oct-20,Burnley,Chelsea,0,3,A,0,1,A,...,2.02,1.00,1.83,2.07,1.84,2.10,1.94,2.12,1.84,2.04
2657,E0,31-Oct-20,Liverpool,West Ham,2,1,H,1,1,D,...,2.85,-1.25,1.91,1.99,1.93,2.00,1.94,2.04,1.90,1.99
2658,E0,31-Oct-21,Aston Villa,West Ham,1,4,A,1,2,A,...,2.07,0.25,1.79,2.11,1.78,2.16,1.90,2.17,1.82,2.07


In [15]:
# Add team ELO feature from master to all_years
joined = all_years.merge(
    master,
    on=["Date", "HomeTeam", "AwayTeam"],
    suffixes=("", "_master"),
)
joined[["HI", "AI"]] = np.nan
joined

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AwayPlayerRatingStd,TeamEloWinProb,PlayerEloDrawProb,PlayerEloWinProb,HomeEloMomentum,AwayEloMomentum,HomeRankMomentum,AwayRankMomentum,HI,AI
0,E0,01-Apr-18,Chelsea,Tottenham,1,3,A,1,1,D,...,0.314261,0.400560,0.330000,0.330000,62.887573,-12.539673,-2.0,1.0,NaN,NaN
1,E0,01-Apr-18,Arsenal,Stoke,3,0,H,0,0,D,...,0.209960,0.757401,0.330000,0.330000,20.978882,7.470215,0.0,-2.0,NaN,NaN
2,E0,01-Apr-19,Arsenal,Newcastle,2,0,H,1,0,H,...,0.269946,0.762502,0.264235,0.583491,-44.682373,-34.069458,0.0,0.0,NaN,NaN
3,E0,01-Apr-23,Chelsea,Aston Villa,0,2,A,0,1,A,...,0.211743,0.633912,0.283756,0.345526,-3.108887,-16.866943,0.0,2.0,NaN,NaN
4,E0,01-Apr-23,Brighton,Brentford,3,3,D,2,2,D,...,0.298958,0.551750,0.245072,0.555928,-5.924194,-3.851929,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,E0,31-Oct-20,Sheffield United,Man City,0,1,A,0,1,A,...,0.293777,0.151418,0.100073,0.050099,9.968994,-5.697266,-3.0,0.0,NaN,NaN
2656,E0,31-Oct-20,Burnley,Chelsea,0,3,A,0,1,A,...,0.198698,0.303893,0.265946,0.220183,18.240723,-22.960083,-3.0,-1.0,NaN,NaN
2657,E0,31-Oct-20,Liverpool,West Ham,2,1,H,1,1,D,...,0.214996,0.811365,0.252909,0.552254,-0.649536,-67.574219,0.0,4.0,NaN,NaN
2658,E0,31-Oct-21,Aston Villa,West Ham,1,4,A,1,2,A,...,0.249537,0.366170,0.278966,0.309205,-4.289429,-14.317749,-1.0,1.0,NaN,NaN


In [16]:
# Length of a season is 380 games
joined = utils.backfill_szn_match_importance(
    joined,
    0,
    380,
    10
)
joined.head(20)

INFO:utils:Match 0 done: Chelsea vs Tottenham, 0.30000000000000004, 0.4
INFO:utils:Match 1 done: Arsenal vs Stoke, 0.0, 0.0
INFO:utils:Match 2 done: Arsenal vs Newcastle, 0.0, 0.1
INFO:utils:Match 3 done: Chelsea vs Aston Villa, 0.0, 0.0
INFO:utils:Match 4 done: Brighton vs Brentford, 0.0, 0.0
INFO:utils:Match 5 done: Bournemouth vs Fulham, 0.0, 0.0
INFO:utils:Match 6 done: Man City vs Liverpool, 0.1, 0.30000000000000004
INFO:utils:Match 7 done: Nott'm Forest vs Wolves, 0.0, 0.1
INFO:utils:Match 8 done: Arsenal vs Leeds, 0.0, 0.0
INFO:utils:Match 9 done: Crystal Palace vs Leicester, 0.0, 0.0
INFO:utils:Match 10 done: Southampton vs Man United, 0.0, 0.0
INFO:utils:Match 11 done: Man City vs Bournemouth, 0.09999999999999998, 0.0
INFO:utils:Match 12 done: Leicester vs Watford, 0.0, 0.0
INFO:utils:Match 13 done: Newcastle vs West Ham, 0.0, 0.0
INFO:utils:Match 14 done: Huddersfield vs Brighton, 0.0, 0.0
INFO:utils:Match 15 done: Crystal Palace vs Burnley, 0.0, 0.0
INFO:utils:Match 16 done:

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AwayPlayerRatingStd,TeamEloWinProb,PlayerEloDrawProb,PlayerEloWinProb,HomeEloMomentum,AwayEloMomentum,HomeRankMomentum,AwayRankMomentum,HI,AI
0,E0,01-Apr-18,Chelsea,Tottenham,1,3,A,1,1,D,...,0.314261,0.400560,0.330000,0.330000,62.887573,-12.539673,-2.0,1.0,0.3,0.4
1,E0,01-Apr-18,Arsenal,Stoke,3,0,H,0,0,D,...,0.209960,0.757401,0.330000,0.330000,20.978882,7.470215,0.0,-2.0,0.0,0.0
2,E0,01-Apr-19,Arsenal,Newcastle,2,0,H,1,0,H,...,0.269946,0.762502,0.264235,0.583491,-44.682373,-34.069458,0.0,0.0,0.0,0.1
3,E0,01-Apr-23,Chelsea,Aston Villa,0,2,A,0,1,A,...,0.211743,0.633912,0.283756,0.345526,-3.108887,-16.866943,0.0,2.0,0.0,0.0
4,E0,01-Apr-23,Brighton,Brentford,3,3,D,2,2,D,...,0.298958,0.551750,0.245072,0.555928,-5.924194,-3.851929,0.0,0.0,0.0,0.0
5,E0,01-Apr-23,Bournemouth,Fulham,2,1,H,0,1,A,...,0.240494,0.393335,0.270790,0.261176,-18.405518,-6.597900,0.0,0.0,0.0,0.0
6,E0,01-Apr-23,Man City,Liverpool,4,1,H,1,1,D,...,0.213771,0.650146,0.240820,0.567682,-24.692749,4.621582,0.0,-1.0,0.1,0.3
7,E0,01-Apr-23,Nott'm Forest,Wolves,1,1,D,1,0,H,...,0.244913,0.411343,0.273614,0.454819,7.678345,0.856201,-2.0,-1.0,0.0,0.1
8,E0,01-Apr-23,Arsenal,Leeds,4,1,H,1,0,H,...,0.185380,0.786238,0.122933,0.808423,-19.110352,-16.219360,1.0,1.0,0.0,0.0
9,E0,01-Apr-23,Crystal Palace,Leicester,2,1,H,0,0,D,...,0.302172,0.471743,0.267066,0.483685,6.169067,24.473877,0.0,-2.0,0.0,0.0
